# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **classification. Because what we will predict is the target label 'passed', which is binary(yes or no). 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = sum(student_data['passed'] == 'yes')


# TODO: Calculate failing students
n_failed = sum(student_data['passed'] == 'no')

# TODO: Calculate graduation rate
grad_rate = 100.0 * n_passed / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
X_all.head

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


<bound method DataFrame.head of      school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  \
0          1.0        0.0    1.0    0.0   18        0.0        1.0   
1          1.0        0.0    1.0    0.0   17        0.0        1.0   
2          1.0        0.0    1.0    0.0   15        0.0        1.0   
3          1.0        0.0    1.0    0.0   15        0.0        1.0   
4          1.0        0.0    1.0    0.0   16        0.0        1.0   
5          1.0        0.0    0.0    1.0   16        0.0        1.0   
6          1.0        0.0    0.0    1.0   16        0.0        1.0   
7          1.0        0.0    1.0    0.0   17        0.0        1.0   
8          1.0        0.0    0.0    1.0   15        0.0        1.0   
9          1.0        0.0    0.0    1.0   15        0.0        1.0   
10         1.0        0.0    1.0    0.0   15        0.0        1.0   
11         1.0        0.0    1.0    0.0   15        0.0        1.0   
12         1.0        0.0    0.0    1.0   15        0.0   

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=66)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

Decision Trees

In industry, a example is to predict C-Section Risk using medical records of 1000 women.[reference](http://www4.stat.ncsu.edu/~dickey/Analytics/Datamine/Reference%20Papers/machine%20learning.pdf)
Strength of decision tree model is:

1.it can deal with both categorical and numerical features

2.the model is simple to understand and people can interpret the model easily by printing the tree to see how it make decisions(while box model).

3.it is fast to train so that it can deal with large dataset.

4.it has no language bias(restriction bias), in other words, the hypothesis space is extremely large. A decision tree can express complicated pattern such as XOR. 

5.it is robust to noise, by setting proper stopping conditions.

Weakness:

1.it has a large search bias(preference), say, you choose the first node with the highest IG, it is a greedy approach, which may fall into local minimum.In other word, it is hard to find the optimal decision tree.

2.it often runs into overfitting. If a tree is too deep and number of training data is small, it can actually predict every training points by memerize them.

3.If you do not do feature transformation, the tree need to be very deep in order to express the combination of features.

4.Considering bias and variance together, a decision tree may not have a best performance compared with other model. That's to say, a deep tree can easily overfit the data, and overfitting may happens very early(say, depth=4), and at that depth, a tree is not complex enough and underfit the data(compare with other model)


for this problem:

1.the features are all transformed into binary or numeric. Note that for numeric variables, such as age, freetime, the values are all integers. This will make the decision tree easy to choose node. That's to say, you only need to try finite(for freetime feature, 5 times) number of spliting. 


Gaussian Naive Bayes (GaussianNB)

applications:[spam detection](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) [automatic medical diagnosis](http://www.research.ibm.com/people/r/rish/papers/RC22230.pdf)

strength:

1.very quick to train, O(size of data) time complexity.

weakness:

1.it relys on the assumption that features are independent with each other. In most cases, the assumption is not true, and the model can learn something wrong.

2.for n features, you should consider n^2 pair of independence.That is bad when you do not have sufficient data. 
http://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes

for this project:

1.Typically when I do supervise learning, I try NB first and use it as baseline.

2.'freetime' and 'goout' are not independent. If a student has little freetime, it is more likely that he seldom go out with friends. In this case, the assumption of NB is not satisfied, and the model will learn something that is misleading.

Support Vector Machines (SVM)

1.PCA+SVM can do facial recognition well. [example](http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html)

2.SVM can do [text and hypertext categorization and hand-written characters recognition](https://en.wikipedia.org/wiki/Support_vector_machine) 

3.[pattern recognitiom, medical diagnostics and object detection.](https://link.springer.com/chapter/10.1007/3-540-26981-9_41)

strength:

1.do well when two class has a clear boundary(margin is large), even if the boundary is complex.

weakness:

1.weak when data is noisy, in which case the boundary is not clear.

2.kernel function and distance function should be well designed(sellected) according to the data and problem. In other word, it depends on how you use domain knowledge.And you should try grid search exhaustedly.

for this project:

1.The dataset has a small number of sample(390), which is good to apply SVM.

2.The students who pass the final exam are usually have an obviously different pattern with those do not pass the exam. In intuition, the margin of the two class tend to be clear.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = svm.SVC(random_state=99)
clf_C = DecisionTreeClassifier(random_state=10)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

clf_group = [clf_A, clf_B, clf_C]
X_train_group = [X_train_100, X_train_200, X_train_300]
y_train_group = [y_train_100, y_train_200, y_train_300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in clf_group:
    for X, y in zip(X_train_group, y_train_group):
        train_predict(clf, X, y, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0014 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8662.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7391.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0037 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8399.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7714.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8141.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7121.
Training a SVC using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 0.9032.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.7871.
Training a SVC using a training set size of 200. . .
Trained model in 0.0045 seconds
Ma

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0017                     |   0.0003                     |       0.8662           |          0.7391       |
| 200               |        0.0009           |        0.0003                 |      0.8399            |              0.7714   |
| 300               |           0.0011              |        0.0003                 |          0.8141        |    0.7121      |

** Classifer 2 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100            |    0.0016                     |         0.0010               |      0.9032            |              0.7871    |
| 200               |     0.0040            |    0.0014                    |       0.8958           |               0.7724  |
| 300               |       0.0081                  |      0.0020                  |   0.8715               |    0.7947    |

** Classifer 3 - DecisionTreeClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0009                      |       0.0002                 |       1.0000           |         0.6500        |
| 200               |       0.0014                  |        0.0002                |      1.0000            |                0.6087 |
| 300               |     0.0023                    |       0.0002                 |     1.0000             |       0.6452          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **I would like to choose SVC as the best model. As the experiment above, SVC has a higher F1 score on testing set than other two models, so it made a better prediction on testing data. The available data has a small size, so it is suitable for SVC to learn. Also, SVC model has many hyperparameters that can be tuned. After I carry on grid search, I expect the F1 score(on testing set) is higher than using default hyperparameters. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **Look at the points of training data, some are red and others are black.Red means the student passed the exam; while black means one did not.There is a gap between red points and black points. After seeing those points, SVC can generate lines or curves just in the middle of the gap, say, there are black points on the left of the line and there are red points on the right. For each point in testing set, if it falls on the left of the lines, SVC will make a prediction of black; if it falls on the right of the line, SVC will make a prediction of red.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [28]:
# TODO: Import 'GridSearchCV' and 'make_scorer'

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
 
parameters = [{'kernel': ['poly'],'degree':[1,2,3,4,5,10], 'C': [0.1,5,8,9,10,11,12,100]},{'kernel': ['sigmoid'], 'C':[0.01,0.09,0.08,0.1,1,10], 'gamma':[0.01,0.09,0.08,0.1,1,10],'coef0':[-0.9,-1,-1.1,0,1]}]
#parameters = [{'kernel': ['poly'],'degree':[1,2,3,4,5,10], 'C': [0.1,5,8,9,10,11,12,100]}]

# TODO: Initialize the classifier

clf= svm.SVC(random_state=87)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, scoring=f1_scorer, param_grid=parameters, cv=3)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)
print(grid_obj.best_estimator_.get_params)
# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

<bound method SVC.get_params of SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=1, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=87, shrinking=True,
  tol=0.001, verbose=False)>
Made predictions in 0.0041 seconds.
Tuned model has a training F1 score of 0.8048.
Made predictions in 0.0013 seconds.
Tuned model has a testing F1 score of 0.7975.


Here, because decision tree overfits the data as default, I also do grid search on it to see what f1 score it can achieve.

In [9]:


from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'max_depth':[2,3,4,5,6],'min_samples_split':[1,2,3,4,5,6],'max_leaf_nodes':[2,3,4,5],'max_features':[39,40,41,42,43,44,45,46,47,48]}
 
# TODO: Initialize the classifier
clf = DecisionTreeClassifier(random_state=86)


# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, scoring=f1_scorer, param_grid=parameters, cv=3)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)
print(grid_obj.best_estimator_.get_params)
# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

<bound method DecisionTreeClassifier.get_params of DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=40, max_leaf_nodes=2, min_samples_leaf=1,
            min_samples_split=1, min_weight_fraction_leaf=0.0,
            presort=False, random_state=86, splitter='best')>
Made predictions in 0.0002 seconds.
Tuned model has a training F1 score of 0.8265.
Made predictions in 0.0002 seconds.
Tuned model has a testing F1 score of 0.7626.



### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

SVC:

      train    test

default:0.8715	0.7947

tuned  :0.8048 0.7975

After tuning, test score increase slightly, training score decrease much. And training score and tesing score is very very close. These illustrate the tuned model reduce overfitting compare to untuned model.

Decision Tree:

      train   test
      
default:1.0000	0.6452

tuned:  0.8265 0.7626

Before tuning, the model overfitted the data greatly, as trainig score = 1 and testing score is not very high. After tuning, the testing score increase much and training score decrease much and these two go quite closely. This illustrates the model after tuning has smaller variance than untuned model.In this case, the grid search make difference!

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.